In [1]:
import os
import random
import math

import numpy as np
import torch
from torch import nn
from tqdm.auto import tqdm

from dataloader import get_dataloader
from resnet import ResNet

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


wandb: Currently logged in as: eddiezhuang. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.init(
    project="resnet",
    config={
        "batch_size": 128,
        "learning_rate": 0.1,
        "iterations": 64000,
        "weight_decay": 0.0001,
        "momentum": 0.9,
        "n": 3,
    },
)

config = wandb.config

In [9]:
train_dataloader, val_dataloader = get_dataloader(True, config.batch_size)
model = ResNet(config.n).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay,
    momentum=config.momentum,
)

iter = 0
while iter < config.iterations:
    model.train()

    for step, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        train_loss = loss_func(outputs, labels)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        accuracy = (outputs.argmax(1) == labels).float().mean()
        metrics = {
            "train/error": 1 - accuracy,
            "train/train_loss": train_loss,
        }
        wandb.log(metrics)

        iter += 1
        if iter >= config.iterations:
            break

Files already downloaded and verified
